# Scrapeo de las competiciones de salto

In [2]:
# Importamos las librerías necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
from selenium.webdriver.support.ui import WebDriverWait
# import xlrd
# from src.soporte_scrapeo_general import get_competiciones, cambio_pestaña, resultados_disciplina, buscador_elementos, guardado_info, competiciones_año, obtencion_año, creacion_dictios_guardado
# from src.soporte_scrapeo_salto import extraccion_resultados_jinetes_caballos

In [2]:
# URL inicial
url = "https://rfhe.com/competiciones/"

In [3]:
# inicializamos el driver y lo abrimos
driver = get_competiciones(url)

In [4]:
# creamos los diccionarios en los que vamos a ir guardando toda la información
dictio_concursos, dictio_pruebas, dictio_jinetes, dictio_caballos = creacion_dictios_guardado()

In [5]:
# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, disciplina = "salto")

In [6]:
ambito_buscado, disciplina_buscada

('nacional', 'salto')

In [7]:
# Buscamos los concursos de febrero del año que nos aparece
competiciones_año(driver)

In [8]:
# Sacamos el total de concursos que hay en el mes de marzo, que compondrá el total de veces que tendrá que iterar el bucle for siguiente
maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[2]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
maximo_n_concursos


40

In [9]:
# Buscamos el primer concurso del mes de marzo
i = 5
selector = f"/html/body/form/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[4]/a"
concurso = buscador_elementos(driver, selector).click()

In [10]:
cambio_pestaña(2,driver)

In [11]:
# Sacamos el contenido general del concurso
contenido_general =  buscador_elementos(driver, "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
contenido_general

['Nombre: CSN2* RACE',
 'Categoría: CSN**',
 'Provincia: Madrid',
 'Localidad: R.A.C.E.',
 'Disciplina: Salto de Obstáculos',
 'Fechas: 01 al 02 de Febrero de 2025',
 'Federación: Federación Hípica de Madrid',
 'Avance: Disponible en PDF  ',
 '  Fechas incripción: 15/Ene->27/Ene/2025',
 'Inscripción: Concurso Finalizado',
 'Admitidos: Publicada  Ver',
 'Resultados: Ver resultados  Ver']

In [12]:
# Guardamos la información del concurso en el diccionario creado
guardado_info(dictio_concursos, contenido_general, info = "concursos", ambito = ambito_buscado)

In [13]:
dictio_concursos

{'Nombre': ['CSN2* RACE'],
 'Categoría': ['CSN**'],
 'Provincia': ['Madrid'],
 'Localidad': ['R.A.C.E.'],
 'Disciplina': ['Salto de Obstáculos'],
 'Federación': ['Federación Hípica de Madrid'],
 'Resultados': ['Ver'],
 'País': ['España']}

In [14]:
# Nos vamos a las pruebas del concurso
path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[20]/td[3]/a"
pruebas = buscador_elementos(driver, path_boton_pruebas).click()
cambio_pestaña(3,driver)

In [15]:
# Obtenemos la información del concurso que se encuentra en la tabla de las pruebas
path_info_restante = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td"
info_concurso_restante = buscador_elementos(driver, path_info_restante).text.split("\n")

# Creamos las nuevas claves que sacamos de la info que esta donde las pruebas
dictio_concursos[info_concurso_restante[2].strip(":")] = []
dictio_concursos[info_concurso_restante[4].strip(":")] = []
dictio_concursos[info_concurso_restante[8].strip(":")] = []

In [16]:
# metemos la info del concurso que nos falta
guardado_info(diccionario = dictio_concursos, elementos = info_concurso_restante, claves = ["Inicio", "Final", "Ámbito"], indices = [3, 5, -1], info = "general")

In [17]:
dictio_concursos

{'Nombre': ['CSN2* RACE'],
 'Categoría': ['CSN**'],
 'Provincia': ['Madrid'],
 'Localidad': ['R.A.C.E.'],
 'Disciplina': ['Salto de Obstáculos'],
 'Federación': ['Federación Hípica de Madrid'],
 'Resultados': ['Ver'],
 'País': ['España'],
 'Inicio': ['01/02/2025'],
 'Final': ['02/02/2025'],
 'Ámbito': ['Nacional']}

In [18]:
# Obtenemos el nombre del concurso para luego meterlo en la tabla de pruebas
path_nombre_concurso = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td/div/div[1]/div/table/tbody/tr/td[2]/table/tbody/tr"
nombre_concurso = buscador_elementos(driver, path_nombre_concurso).text.strip()
nombre_concurso

'CSN2* RACE'

In [19]:
# Obtenemos las pruebas del concurso
path_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]"
pruebas = buscador_elementos(driver, path_pruebas).text.split("\n")
pruebas

['Disciplina',
 'Fecha',
 'Prueba',
 'Categoría',
 'Número',
 'Resultados',
 'Salto de Obstáculos',
 '01/02/2025',
 '1,10 - Acc (238.2.1)',
 'CSN**',
 '1',
 'Disponibles',
 'Salto de Obstáculos',
 '01/02/2025',
 '1,20 - A c.c. (238.2.1)',
 'CSN**',
 '2',
 'Disponibles',
 'Salto de Obstáculos',
 '01/02/2025',
 '1,30 - A c.c. (238.2.1)',
 'CSN**',
 '3',
 'Disponibles',
 'Salto de Obstáculos',
 '02/02/2025',
 '1,25 - A c.c. (238.2.1)',
 'CSN**',
 '4',
 'Disponibles',
 'Salto de Obstáculos',
 '02/02/2025',
 '1,35 GP - A.c.c. y desempate (238.2.2)',
 'CSN**',
 '5',
 'Disponibles',
 'Salto de Obstáculos',
 '02/02/2025',
 '1,15 - A c.c. (238.2.1)',
 'CSN**',
 '6',
 'Disponibles']

In [ ]:
claves_pruebas = list(dictio_pruebas.keys())
guardado_info(diccionario = dictio_pruebas,elementos = pruebas, claves = claves_pruebas, indices = [6, 7, 8, 9, 10], step = 6, guardado = False, info = "general")

numero_pruebas = len(pruebas[6::6])  # Asumimos que 'Disciplina' tiene una fila por prueba
dictio_pruebas["Concurso"].extend([nombre_concurso] * numero_pruebas)

In [20]:
# accedemos a los resultados de la primera prueba
path_resultados_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]/div[2]/div/table/tbody/tr/td/div/div/table/tbody/tr[1]/td/div/div/table/tbody/tr[1]/td/div/div[3]/div/table/tbody/tr/td/a"
resultados_pruebas = buscador_elementos(driver, path_resultados_pruebas).click()

In [ ]:
# url inicial
url = "https://rfhe.com/competiciones/"


# inicializamos el driver y lo abrimos
driver = get_competiciones(url)

dictio_concursos_completo_nac, dictio_pruebas_completo_nac, dictio_jinetes_completo_nac, dictio_caballos_completo_nac = creacion_dictios_guardado()

time.sleep(8)

# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, disciplina = "completo")

time.sleep(5)

año = obtencion_año(driver)

while año >= 2024:

    # Buscamos los concursos de febrero del año que nos aparece
    competiciones_año(driver)

    time.sleep(3)
    # Sacamos el total de concursos que hay en el mes de febrero, que compondrá el total de veces que tendrá que iterar el bucle for siguiente

    maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[2]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))

    for i in range(5, maximo_n_concursos + 5):
        try:
            time.sleep(4)
            # clickamos en el concurso
            selector = f"/html/body/form/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[4]/a"
            time.sleep(1)
            concurso = buscador_elementos(driver, selector).click()

            time.sleep(5)

            # Sacamos el contenido general del concurso
            contenido_general =  buscador_elementos(driver, "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
            
            if i == 5 and año == 2025:

                # Guardamos la información del concurso en el diccionario creado
                guardado_info(dictio_concursos_completo_nac, contenido_general, info = "concursos", ambito = ambito_buscado)

                # Nos vamos a las pruebas del concurso
                path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[20]/td[3]/a" 
                pruebas = buscador_elementos(driver, path_boton_pruebas).click()
                cambio_pestaña(2,driver)

                time.sleep(3)

                # Obtenemos la información del concurso que se encuentra en la tabla de las pruebas
                path_info_restante = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td"
                info_concurso_restante = buscador_elementos(driver, path_info_restante).text.split("\n")

                # Creamos las nuevas claves que sacamos de la info que esta donde las pruebas
                dictio_concursos_completo_nac[info_concurso_restante[2].strip(":")] = []
                dictio_concursos_completo_nac[info_concurso_restante[4].strip(":")] = []
                dictio_concursos_completo_nac[info_concurso_restante[8].strip(":")] = []
                
                # metemos la info del concurso que nos falta
                guardado_info(diccionario = dictio_concursos_completo_nac, elementos = info_concurso_restante, claves = ["Inicio", "Final", "Ámbito"], indices = [3, 5, -1], info = "general")

                # Obtenemos el nombre del concurso para luego meterlo en la tabla de pruebas
                path_nombre_concurso = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td/div/div[1]/div/table/tbody/tr/td[2]/table/tbody/tr"
                nombre_concurso = buscador_elementos(driver, path_nombre_concurso).text.strip()

                # Obtenemos las pruebas del concurso
                path_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]"
                pruebas = buscador_elementos(driver, path_pruebas).text.split("\n")
                
                claves_pruebas = list(dictio_pruebas_completo_nac.keys())
                guardado_info(diccionario = dictio_pruebas_completo_nac, elementos = pruebas, claves = claves_pruebas, indices = [6, 7, 8, 9, 10], step = 6, guardado = False, info = "general")

                numero_pruebas = len(pruebas[6::6])  # Asumimos que 'Disciplina' tiene una fila por prueba
                dictio_pruebas_completo_nac["Concurso"].extend([nombre_concurso] * numero_pruebas)

                # accedemos a los resultados de la primera prueba
                path_resultados_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]/div[2]/div/table/tbody/tr/td/div/div/table/tbody/tr[1]/td/div/div/table/tbody/tr[1]/td/div/div[3]/div/table/tbody/tr/td/a"
                resultados_pruebas = buscador_elementos(driver, path_resultados_pruebas).click()

                time.sleep(4)

                extraccion_resultados_jinetes_caballos(driver, dictio_jinetes_completo_nac, dictio_caballos_completo_nac)

                driver.close()
                cambio_pestaña(1,driver)

            if i == 5 and año < 2025:

                # Guardamos la información del concurso en el diccionario creado
                guardado_info(dictio_concursos_completo_nac, contenido_general, info = "concursos", ambito = ambito_buscado)

                # Nos vamos a las pruebas del concurso
                path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[20]/td[3]/a" 
                pruebas = buscador_elementos(driver, path_boton_pruebas).click()
                cambio_pestaña(2,driver)

                time.sleep(3)

                # Obtenemos la información del concurso que se encuentra en la tabla de las pruebas
                path_info_restante = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td"
                info_concurso_restante = buscador_elementos(driver, path_info_restante).text.split("\n")
                
                # metemos la info del concurso que nos falta
                guardado_info(diccionario = dictio_concursos_completo_nac, elementos = info_concurso_restante, claves = ["Inicio", "Final", "Ámbito"], indices = [3, 5, -1], info = "general")

                # Obtenemos el nombre del concurso para luego meterlo en la tabla de pruebas
                path_nombre_concurso = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td/div/div[1]/div/table/tbody/tr/td[2]/table/tbody/tr"
                nombre_concurso = buscador_elementos(driver, path_nombre_concurso).text.strip()

                # Obtenemos las pruebas del concurso
                path_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]"
                pruebas = buscador_elementos(driver, path_pruebas).text.split("\n")
                
                claves_pruebas = list(dictio_pruebas_completo_nac.keys())
                guardado_info(diccionario = dictio_pruebas_completo_nac, elementos = pruebas, claves = claves_pruebas, indices = [6, 7, 8, 9, 10], step = 6, guardado = False, info = "general")

                numero_pruebas = len(pruebas[6::6])  # Asumimos que 'Disciplina' tiene una fila por prueba
                dictio_pruebas_completo_nac["Concurso"].extend([nombre_concurso] * numero_pruebas)

                # accedemos a los resultados de la primera prueba
                path_resultados_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]/div[2]/div/table/tbody/tr/td/div/div/table/tbody/tr[1]/td/div/div/table/tbody/tr[1]/td/div/div[3]/div/table/tbody/tr/td/a"
                resultados_pruebas = buscador_elementos(driver, path_resultados_pruebas).click()

                time.sleep(4)

                extraccion_resultados_jinetes_caballos(driver, dictio_jinetes_completo_nac, dictio_caballos_completo_nac)

                driver.close()
                cambio_pestaña(1,driver)
                
            elif i != 5 and "Resultados: Ver resultados  Ver" in contenido_general:

                # Guardamos la información del concurso en el diccionario creado
                guardado_info(dictio_concursos_completo_nac, contenido_general, info = "concursos", ambito = ambito_buscado)

                # Nos vamos a las pruebas del concurso
                path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[20]/td[3]/a" 
                pruebas = buscador_elementos(driver, path_boton_pruebas).click()
                cambio_pestaña(2,driver)

                time.sleep(3)

                # Obtenemos la información del concurso que se encuentra en la tabla de las pruebas
                path_info_restante = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td"
                info_concurso_restante = buscador_elementos(driver, path_info_restante).text.split("\n")
                
                # metemos la info del concurso que nos falta
                guardado_info(diccionario = dictio_concursos_completo_nac, elementos = info_concurso_restante, claves = ["Inicio", "Final", "Ámbito"], indices = [3, 5, -1], info = "general")

                # Obtenemos el nombre del concurso para luego meterlo en la tabla de pruebas
                path_nombre_concurso = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td/div/div[1]/div/table/tbody/tr/td[2]/table/tbody/tr"
                nombre_concurso = buscador_elementos(driver, path_nombre_concurso).text.strip()

                # Obtenemos las pruebas del concurso
                path_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]"
                pruebas = buscador_elementos(driver, path_pruebas).text.split("\n")
                
                claves_pruebas = list(dictio_pruebas_completo_nac.keys())
                guardado_info(diccionario = dictio_pruebas_completo_nac, elementos = pruebas, claves = claves_pruebas, indices = [6, 7, 8, 9, 10], step = 6, guardado = False, info = "general")

                numero_pruebas = len(pruebas[6::6])  # Asumimos que 'Disciplina' tiene una fila por prueba
                dictio_pruebas_completo_nac["Concurso"].extend([nombre_concurso] * numero_pruebas)

                # accedemos a los resultados de la primera prueba
                path_resultados_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]/div[2]/div/table/tbody/tr/td/div/div/table/tbody/tr[1]/td/div/div/table/tbody/tr[1]/td/div/div[3]/div/table/tbody/tr/td/a"
                resultados_pruebas = buscador_elementos(driver, path_resultados_pruebas).click()
                
                time.sleep(4)

                extraccion_resultados_jinetes_caballos(driver, dictio_jinetes_completo_nac, dictio_caballos_completo_nac)

                driver.close()
                cambio_pestaña(1,driver)

            # aqui tenemos los concursos que han sido suspendidos
            elif i != 5 and "Resultados: Ver resultados  Ver" not in contenido_general:
                    continue
            
        # Manejamos las competiciones que han sido aplazadas, las cuales no tendremos en cuenta
        except NoSuchElementException:

            enlace_concurso = driver.find_element(By.XPATH, f"/html/body/form/div/div/div/div/div[3]/table/tbody/tr[{i}]/td[4]/font")
            print(enlace_concurso.text)
            continue

        except NoSuchElementException:
            driver.refresh()
            continue
        
        driver.back()

    buscador_elementos(driver,"/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/a").click()
                               
            
    año = obtencion_año(driver)

## Concursos nacionales

In [ ]:
# url inicial
url = "https://rfhe.com/competiciones/"

dictio_concursos_salto_nac, dictio_pruebas_salto_nac, dictio_jinetes_salto_nac, dictio_caballos_salto_nac = creacion_dictios_guardado()
urls_resultados = []

driver = get_competiciones(url)
time.sleep(2)

# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, disciplina = "salto")
cambio_pestaña(1,driver)
time.sleep(2)

año = obtencion_año(driver)

while año >= 2017:

    # vamos a todos los concursos del año en el que estemos
    competiciones_año(driver)

    maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[13]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
    if año == 2025:
        rango = range(5,139)
    elif año < 2025:
        rango = range(5, maximo_n_concursos + 5) 

    for i in rango:

        try:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/a")
            print(concurso_bueno.text)
            time.sleep(1)
            try:
                concurso_bueno.click()
                cambio_pestaña(2, driver)
                time.sleep(2)
                contenido_general =  buscador_elementos(driver, "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
                                                                
                if "Resultados: Ver resultados  Ver" not in contenido_general:
                    driver.back()
                    time.sleep(1)

                elif "Resultados: Ver resultados  Ver" in contenido_general:
                    print(contenido_general)
                    
                    extraccion_info_concursos(driver, diccionario_concursos=dictio_concursos_salto_nac, ambito_buscado=ambito_buscado, contenido_general=contenido_general)
                    cambio_pestaña(3, driver)
                    time.sleep(1)

                    if i == 5 and año == 2025:
                        extraccion_info_pruebas(driver, dictio_concursos_salto_nac, dictio_pruebas_salto_nac, urls_resultados, es_primer_concurso=True)
                    else:
                        extraccion_info_pruebas(driver, dictio_concursos_salto_nac, dictio_pruebas_salto_nac, urls_resultados)

                    driver.close()
                    cambio_pestaña(2, driver)
                    driver.close()
                    cambio_pestaña(1, driver)
                    time.sleep(2)

            except NoSuchElementException:
                raise NoSuchElementException

        except NoSuchElementException:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/font").text
            print(concurso_bueno)
    
    buscador_elementos(driver,"/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/a").click()        
    año = obtencion_año(driver)
driver.quit()

## Concursos internacionales

In [ ]:
# url inicial
url = "https://rfhe.com/competiciones/"

dictio_concursos_salto_int, dictio_pruebas_salto_int, dictio_jinetes_salto_int, dictio_caballos_salto_int = creacion_dictios_guardado()
urls_resultados_salto_int = []

# inicializamos el driver y lo abrimos
driver = get_competiciones(url)
time.sleep(2)

# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, ambito = "internacional", disciplina = "salto")
cambio_pestaña(1,driver)
time.sleep(2)

año = obtencion_año(driver)

while año >= 2024:

    # Buscamos los concursos del año que nos aparece
    competiciones_año(driver)

    time.sleep(3)

    maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[13]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
    if año == 2025:
        rango = range(5,33)
    elif año < 2025:
        rango = range(5, maximo_n_concursos + 5) 

    for i in rango:
        
        try:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/a")
            print(concurso_bueno.text)
            time.sleep(1)
            try:
                concurso_bueno.click()
                cambio_pestaña(2, driver)
                time.sleep(2)
                contenido_general = buscador_elementos(driver, "/html/body/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
                                                                
                if "Resultados: Ver resultados  Ver" not in contenido_general:
                    driver.close()
                    cambio_pestaña(1, driver)
                    time.sleep(1) 
                
                elif "Resultados: Ver resultados  Ver" in contenido_general:
                    extraccion_info_concursos(driver, diccionario_concursos=dictio_concursos_salto_int, ambito_buscado=ambito_buscado, contenido_general=contenido_general)
                    cambio_pestaña(3,driver)
                    if i == 5 and año == 2025:
                        extraccion_info_pruebas(driver, dictio_concursos_salto_int, dictio_pruebas_salto_int, urls_resultados_salto_int, es_primer_concurso=True)
                    else:
                        extraccion_info_pruebas(driver, dictio_concursos_salto_int, dictio_pruebas_salto_int, urls_resultados_salto_int)
                    driver.close()
                    cambio_pestaña(2, driver)
                    driver.close()
                    cambio_pestaña(1, driver)
                    time.sleep(2)

            except NoSuchElementException:
                raise NoSuchElementException

        except NoSuchElementException:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/font").text
            print(concurso_bueno)
    
    buscador_elementos(driver,"/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/a").click()
    time.sleep(3)        
    año = obtencion_año(driver)
driver.quit()